# End of semester review lab

**Instructions**

This lab is designed to be a review of some of the key concepts and coding from the second half of the semester.

Your task is to predict the wage in the following data set using all of the available features.  You should do this using two separate models, and compare which model has the lowest RMSE (Root Mean Squared Error).  The two models you should use are linear regression and K-nearest neighbor, with K = 1.  Note that we are using K nearest neighbor in a new way because this is not a classification problem.  The K-nearest neighbor should simply use the nearest neighbor's wage as the prediction wage.

You will need to follow these steps

## 1. Setup
Run the two cells below to load the packages and data.

The dataset CPS85 contains data on 534 individuals surveyed in the year 1985.

    wage = The hourly wage
    educ = years of education
    sex = sex (male or female)
    exper = years of experience
    union = whether or not the person was in a union

In [ ]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline

In [ ]:
CPS85 = Table.read_table("CPS85_small.csv")
CPS85

wage,educ,sex,exper,union
9,10,M,27,Not
5.5,12,M,20,Not
3.8,12,F,4,Not
10.5,12,F,29,Not
15,12,M,40,Union
9,16,F,27,Not
9.57,12,F,5,Union
15,14,M,22,Not
11,8,M,42,Not
5,12,F,14,Not


**Step 1: Data Preparation**
1. You can't fit a model with categorical data the such as sex and union as text.  You will need to convert those columns into numbers.  Fortunately since both are binary variables, you just need to convert M to 1 and F to 0, and Union to 1 and Not to 0.

2. Convert each column to standard units

3. Save the cleaned data as CPS85_clean

4. Create a train/test split with 70% of the data for training and 30% for testing.  Use a random seed of 1234 so that we all get the same results when we do the train test split.  There should be 374 training observations and 160 test observations.  Name the resulting data sets train and test.  




In [ ]:
def sex_to_b(text):
    if text == 'M':
        return 1
    else: return 0

In [ ]:
def union_to_b(text):
    if text == 'union':
        return 1
    else: return 0

In [ ]:
sex_0_1 = CPS85.apply(sex_to_b, 'sex')
union_0_1 = CPS85.apply(union_to_b, 'union')


In [ ]:
def standard_units(arr):
    return arr - np.average(arr)/np.std(arr)

In [ ]:
CPS85_clean = (CPS85
    .with_column('wage', standard_units(CPS85.column('wage')))
    .with_column('educ', standard_units(CPS85.column('educ')))
    .with_column('sex', standard_units(sex_0_1))
    .with_column('exper', standard_units(CPS85.column('exper')))
    .with_column('union', standard_units(union_0_1))
)
CPS85_clean

In [ ]:
np.random.seed(1234)
shuffled = CPS85_clean.sample(with_replacement=False) # Randomly permute the rows
train = shuffled.take(np.arange(374))
test  = shuffled.take(np.arange(374, CPS85_clean.num_rows))

In [ ]:
train

wage,educ,sex,exper,union
8.75,12,1,9,0
12,14,0,10,0
20.4,17,1,3,0
13,12,1,25,0
8.5,12,1,13,0
7,12,0,10,0
4.35,11,0,20,0
6,7,0,15,0
8,12,0,8,0
10,10,0,25,0


In [ ]:
test

wage,educ,sex,exper,union
11,8,1,42,0
3.35,13,0,2,0
7.78,16,0,6,0
6.25,12,0,6,0
13.71,12,1,43,0
3.35,12,0,8,0
13.1,10,0,38,0
18,16,1,38,0
12,18,0,18,0
3.8,12,0,4,0


# Regression Model 

**Step 2: Regression Model** 

1. Define a function that calculates the root mean squared error of a regression model that predicts the wage using all other features.  Note: You have only done this before with one 'x' variable.  Now there are more than one.  All you have to do is to add a 'slope' for each variable. For example the mathematical equation for the predicted value is:

$fitted = slope_1*educ + slope_2*sex + slope_3*exper + slope_4*union + intercept$

Once you have the fitted value for each data point, you can use it to calculate the RMSE for the model

*Hint* This [example from the textbook](https://umass-data-science.github.io/190fwebsite/textbook/15/3/method-of-least-squares/) should help you.  This was also in the last lab we did.

2. Use the minimize function to find the slopes and intercept that minimize the the RMSE.  These slopes and intercepts are your model, and the RMSE that these slopes give you is the *training RMSE of your model*

In [ ]:
def rmse(slope1, slope2, slope3, slope4, intercept):
    e = train.column('educ')
    s = train.column('sex')
    ex = train.column('exper')
    u = train.column('union')
    w = train.column('wage')
    fitted = slope1 * e + slope2 * s + (slope3 * ex + slope4 * u) + intercept
    rmse = np.sqrt(np.mean((fitted - w) ** 2))
    return rmse

In [ ]:
e = train.column('educ')
s = train.column('sex')
ex = train.column('exper')
u = train.column('union')
w = train.column('wage')
fitted = 1 * e + 2 * s + 3* ex + 4 * u + 5
np.sqrt(np.mean((fitted - w)**2))   
    

73.71883214326475

In [ ]:
rmse(1,2,3,4,5)

73.71883214326475

In [ ]:
model_coefs = minimize(rmse)
model_coefs#these "slopes" minimize the RMSE

array([ 9.31258141e-01,  2.14693454e+00,  1.04060094e-01,  3.15191442e+02,
       -6.12851452e+00])

**Step 3: Calculate test set RMSE**

1.  Use the slopes and intercept you calculated from your training data to predict the wage of each observation in your test set.

2.  Calculate the RMSE of those predictions.  This is the *test set RMSE of your model*.

In [ ]:
#define a function to calculate test set RMSE notice we use test set here
def rmse_test(slope1, slope2, slope3, slope4, intercept):
    e = test.column('educ')
    s = test.column('sex')
    ex = test.column('exper')
    u = test.column('union')
    w = test.column('wage')
    fitted = slope1 * e + slope2 * s + (slope3 * ex + slope4 * u) + intercept
    rmse = np.sqrt(np.mean((w - fitted) ** 2))
    return rmse

In [ ]:
model_coefs[0]

0.9312581413414276

In [ ]:
#RMSE of test set
rmse_test(model_coefs[0], model_coefs[1], model_coefs[2], model_coefs[3], model_coefs[4])

4.335290672696314

In [ ]:
#RMSE of training set.
#notice the test RMSE is very close, and a bit smaller this is a little unusual but not 
#so unusual as to worry.
rmse(model_coefs[0], model_coefs[1], model_coefs[2], model_coefs[3], model_coefs[4])

4.48293960279929

# K-nearest Neighbor Model

Note you should use the same training and test data for this model as you did for the regression.

1. Create a function or series of functions that finds the nearest neighbor in the training data of a single row of data.  You will find [this notebook from lecture helpful](http://datahub.cs.umass.edu/hub/user-redirect/git-sync?repo=https://github.com/umass-data-science/materials-sp21&subPath=lec/lec22.ipynb)

*hint:* you want the closest() function to work.  You don't need the majority_class or classify functions because this is a regression problem.








In [ ]:
 def distance(pt1, pt2):
    """Return the distance between two points, represented as arrays"""
    return np.sqrt(sum((pt1 - pt2)**2))

def row_distance(row1, row2):
    """Return the distance between two numerical rows of a table"""
    return distance(np.array(row1), np.array(row2))

def distances(train, example):
    """Compute distance between example and every row in train.
    Return train augmented with Distance column"""
    distances = make_array()
    attributes = train.drop('Class')
    for row in attributes.rows:
        distances = np.append(distances, row_distance(row, example))
    return train.with_column('Distance', distances)

def closest(train, example, k):
    """Return a table of the k closest neighbors to example"""
    return distances(train, example).sort('Distance').take(np.arange(k))

2.  Use you function to find the nearest neighbor (k=1) in the training set of each observation in the test set.  The value of the nearest neighbor's wage in the training set is the prediction of the value of wage for the test observation.

In [ ]:
test.row(0)

Row(wage=11.0, educ=8, sex=1, exper=42, union=0)

In [ ]:
predicted_wage = closest(train, test.row(0), 1)
predicted_wage[0][0]

8.0

In [ ]:
predicted_wage_array = make_array()
for i in np.arange(test.num_rows):
    predicted_wage = closest(train, test.row(i), 1)[0][0]#this chooses the wage of the nearest neighbor
    predicted_wage_array = np.append(predicted_wage_array, predicted_wage)

In [ ]:
predicted_wage_array

array([ 8.  ,  4.25,  6.25,  6.94, 12.5 ,  3.35, 13.16, 17.86, 11.22,
        4.13,  6.25,  7.5 , 19.  ,  4.5 ,  6.  , 11.25,  9.6 ,  6.  ,
        6.  , 11.79, 10.  ,  8.06, 12.  ,  6.5 ,  5.  ,  4.17,  5.25,
        3.6 , 11.25,  8.89, 11.11,  6.25,  6.67, 24.98,  3.95,  5.5 ,
       26.29,  6.94, 10.5 ,  5.5 , 10.  ,  5.5 ,  4.25,  4.  , 19.88,
        8.5 , 11.25,  5.5 ,  5.5 ,  4.  ,  7.  ,  9.33, 20.55,  8.75,
        5.  , 19.  ,  5.  ,  8.  ,  8.  ,  6.  ,  4.  ,  4.5 , 13.45,
       11.25,  8.5 ,  7.61,  4.25, 10.  ,  4.13,  5.55, 15.  ,  4.28,
       24.98,  5.  ,  3.5 , 20.5 , 24.98,  6.4 ,  3.35,  7.5 ,  9.  ,
        4.  ,  9.5 , 19.88, 13.45,  6.  ,  5.  ,  5.25,  5.65,  7.5 ,
        8.49, 12.  ,  4.  ,  5.25,  3.75,  5.  ,  6.88, 11.79, 14.29,
       10.  ,  7.5 ,  5.71,  6.75, 11.25,  4.5 ,  7.5 , 15.95,  4.25,
       14.  ,  8.  , 20.5 ,  8.5 ,  5.2 , 10.28,  8.06, 13.45,  8.5 ,
        7.96, 11.25,  7.61, 11.67,  3.5 ,  7.  ,  6.25,  4.85, 10.  ,
        3.95, 12.22,

3.  Calculate the RMSE of the predicted wage using your nearest neighbor prediction.  This is the RMSE of your nearest neighbor model

In [ ]:
predicted_wage_array - test.column('wage')

array([-3.  ,  0.9 , -1.53,  0.69, -1.21,  0.  ,  0.06, -0.14, -0.78,
        0.33, -0.42, -0.03,  1.  , -0.75, -1.  ,  1.25,  0.6 , -0.36,
        0.38,  0.54,  0.5 ,  0.56, -0.5 ,  1.  ,  0.  , -0.83, -0.25,
       -0.04,  0.64,  1.39,  0.33,  0.25,  1.27,  0.  ,  0.3 , -0.6 ,
        1.31,  0.54, -1.5 ,  0.  ,  1.  ,  0.  ,  0.7 , -0.1 ,  1.38,
       -0.86,  0.93, -0.5 ,  0.1 ,  0.5 ,  0.25, -0.27, -0.7 , -1.25,
        0.  ,  0.84, -0.3 ,  0.  , -6.  , -0.25,  0.  ,  0.15,  0.95,
        4.25,  1.05, -0.04,  0.8 , -1.  , -0.42, -0.55,  0.  , -0.31,
        0.  , -0.77, -0.48, -2.  , -1.02, -1.6 ,  0.  ,  0.92,  1.2 ,
       -0.5 , -0.15,  0.41, -0.76,  0.5 ,  0.  , -0.5 , -0.18,  1.22,
        0.74, -0.57, -0.75,  0.75,  0.15, -1.25,  1.78,  1.17, -1.09,
        0.  ,  1.  ,  0.91,  1.8 ,  0.75,  0.85,  0.  , -1.55, -0.75,
        0.55,  0.62, -1.5 ,  0.07, -0.65,  0.28, -0.14,  0.19, -0.65,
        0.29,  0.82, -0.08,  1.05,  0.  , -1.5 , -0.48,  0.35, -0.67,
        0.2 ,  0.22,

In [ ]:
np.sqrt(np.mean((predicted_wage_array - test.column('wage'))**2))

1.0876361983678182

**Results:** What are the RMSEs of your two models?  Which one has a lower RMSE on the test set? 

RMSE of the linear regression model is 4.48, RMSE of the nearest neighbor model is 13.7, so the linear regression model has a lower test set error